In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

**LangChain** 은 직렬화(Serialization) 체계를 공유합니다.

`is_lc_serializable` 클래스 메서드로 실행하여 LangChain 클래스가 직렬화 가능한지 확인할 수 있습니다.


In [11]:
from langchain_openai import ChatOpenAI
from langchain.llms.loading import load_llm

In [10]:
# 직렬화가 가능한지 체크합니다.
print(f"ChatOpenAI: {ChatOpenAI.is_lc_serializable()}")

ChatOpenAI: True


In [6]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [7]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{fruit}의 색상이 무엇입니까?")
prompt

PromptTemplate(input_variables=['fruit'], template='{fruit}의 색상이 무엇입니까?')

In [8]:
chain = prompt | llm

In [9]:
chain.is_lc_serializable()

True

## dumpd: 모델 저장

직렬화 가능한 모든 객체는 딕셔너리 또는 json 문자열로 직렬화할 수 있습니다.


In [12]:
from langchain.load import dumpd

dumped_chain = dumpd(chain)
dumped_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string',
    'partial_variables': {}},
   'name': 'PromptTemplate',
   'graph': {'nodes': [{'id': 0,
      'type': 'schema',
      'data': {'title': 'PromptInput',
       'type': 'object',
       'properties': {'fruit': {'title': 'Fruit', 'type': 'string'}}}},
     {'id': 1,
      'type': 'runnable',
      'data': {'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
       'name': 'PromptTemplate'}},
     {'id': 2,
      'type': 'schema',
      'data': {'title': 'PromptTemplateOutput',
       'anyOf': [{'$ref': '#/definitions/StringPromptValue'},
        {'$ref': '#/definitions/ChatPromptValueConcrete'}],
       'definitions': {'StringPromptValue': {'title'

In [19]:
import pickle

# fuit_chain.pkl 파일로 직렬화된 체인을 저장합니다.
with open("fruit_chain.pkl", "wb") as f:
    pickle.dump(dumped_chain, f)

## load: 저장한 모델 불러오기


In [21]:
import pickle

with open("fruit_chain.pkl", "rb") as f:
    loaded_chain = pickle.load(f)

loaded_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string',
    'partial_variables': {}},
   'name': 'PromptTemplate',
   'graph': {'nodes': [{'id': 0,
      'type': 'schema',
      'data': {'title': 'PromptInput',
       'type': 'object',
       'properties': {'fruit': {'title': 'Fruit', 'type': 'string'}}}},
     {'id': 1,
      'type': 'runnable',
      'data': {'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
       'name': 'PromptTemplate'}},
     {'id': 2,
      'type': 'schema',
      'data': {'title': 'PromptTemplateOutput',
       'anyOf': [{'$ref': '#/definitions/StringPromptValue'},
        {'$ref': '#/definitions/ChatPromptValueConcrete'}],
       'definitions': {'StringPromptValue': {'title'

In [22]:
from langchain.load.load import load


loaded_chain = load(loaded_chain)

In [23]:
print(loaded_chain.invoke({"fruit": "사과"}))

content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.'
